In [1]:
import requests
import csv
from fake_useragent import UserAgent
from lxml import html

In [2]:
# Парсим https://film.arjlover.net/film/
url_catalogue = 'https://film.arjlover.net'
url_category = '/film/'
ua = UserAgent()
headers = {"User-Agent": ua.chrome}
session = requests.session()
response = session.get(f'{url_catalogue}{url_category}', headers=headers)

In [3]:
# Находим таблицу, в которой содержатся строки с классом "e" или "o"
tree = html.fromstring(response.content)
tables = tree.xpath('//table[.//tr[@class="e"] or .//tr[@class="o"]]')
if len(tables):
    table = tables[0]
else:
    print('Таблица для парсинга не найдена!')

In [4]:
# Парсим заголовки колонок из первого <tr>, удаляем лишние пробелы
header_row = [header.strip() for header in table.xpath('//tr[@class="head"]/th/text()')][0:-1]

# Вместо столбца "Скачать" делаем два отдельных
header_row += ['http', 'torrent']
print(header_row)

['N', 'Название', 'Размер', 'Картинка', 'Длит-ть', 'http', 'torrent']


In [5]:
# Парсим строки с данными
rows = tree.xpath('//tr[@class="o" or @class="e"]')
data = []

# Парсим строки
for row in rows:
    row_data = {}
    
    # Извлекаем значение N из первой ячейки, включая текст в тегах <a> и <td>
    cell = row.xpath('./td[1]//text()')
    row_data['N'] = int(''.join(cell).strip()) if cell else None
    
    # Извлекаем значения остальных полей
    cell = row.xpath('./td[2]//text()')
    row_data['Название'] = cell[0].strip() if cell else None

    cell = row.xpath('./td[3]//text()')
    row_data['Размер'] = cell[0].strip() if cell else None

    cell = row.xpath('./td[4]//text()')
    row_data['Картинка'] = cell[0].strip() if cell else None

    cell = row.xpath('./td[5]//text()')
    row_data['Длит-ть'] = cell[0].strip() if cell else None
    
    # Для столбца "Скачать" формируем полные ссылки (кроме ed2k)
    cell = row.xpath('./td[6]//a/@href')
    download_links = cell if cell else []
    row_data['http'] = f'{url_catalogue}{download_links[0]}' if len(download_links) else ''
    row_data['torrent'] = f'{url_catalogue}{download_links[2]}' if len(download_links) > 1 else ''
    
    data.append(row_data)

# Отладка (должно быть 6174)
print(len(data))
for row in data[0:3]:
    print(row)

6174
{'N': 1, 'Название': '01-99', 'Размер': '326.578.176', 'Картинка': '720x544', 'Длит-ть': '00:20:59', 'http': 'https://film.arjlover.net/film/01.99.avi', 'torrent': 'https://film.arjlover.net/torrents/01.99.avi.torrent'}
{'N': 2, 'Название': '126 слов', 'Размер': '701.581.312', 'Картинка': '720x544', 'Длит-ть': '00:49:05', 'http': 'https://film.arjlover.net/film/126.slov.avi', 'torrent': 'https://film.arjlover.net/torrents/126.slov.avi.torrent'}
{'N': 3, 'Название': '1812 год', 'Размер': '610.891.776', 'Картинка': '700x528', 'Длит-ть': '00:32:44', 'http': 'https://film.arjlover.net/film/1812.god.avi', 'torrent': 'https://film.arjlover.net/torrents/1812.god.avi.torrent'}


In [6]:
# Записываем данные в CSV-файл
with open('homework04_films.csv', newline='', mode='w', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=header_row, delimiter=';')        
    writer.writeheader()  
    writer.writerows(data)         

# Выводим результат
print(f'Данные успешно сохранены!')

Данные успешно сохранены!
